In [44]:
import pandas as pd
import numpy as np

In [45]:
import openpyxl
from openpyxl import Workbook
from openpyxl.styles import Alignment
from datetime import datetime
from copy import copy

## Extract data from source file:

In [46]:
df = pd.read_excel(
    io='C:\\Users\\jio\\Desktop\\VCT8.xlsx',
    header=0
)

# Add column 'STT' and reindex columns:
for idx in range(len(df['Số CONTAINER'])):
    df.loc[idx, ['STT']] = idx + 1

columns = list(df.columns)
temp = columns[-1]
for idx in range(len(columns)-1, -1, -1):
    columns[idx] = columns[idx - 1]

columns[0] = temp
df = df.reindex(columns=columns) 
df['STT'] = df['STT'].astype('int8')

## Customer table:
- Modify the main table:

In [48]:
LDP = df.drop(
    columns=['Kích cỡ', 'Khách hàng ', 'Số xe ', 'Tạm ứng', 'Tổng cộng (VC)', 'Tổng cộng (NH)']
)

# Create and input data into column 'Tổng cộng':
LDP['Tổng cộng'] = [0 for i in range(len(LDP['Số CONTAINER']))]

for idx in range(len(LDP['Số CONTAINER'])):
    LDP.loc[idx, ['Tổng cộng']] = LDP.iloc[idx, [7, 8, 10, 11, 12]].sum()

# Export to xlsx file and finish part I:
LDP.to_excel(
    'C:\\Users\\jio\\Desktop\\test.xlsx',
    header=True,
    index=False,
    startrow=8
)

- Add in details:

In [49]:
# Open the xlsx file:
file = openpyxl.load_workbook(
    'C:\\Users\\jio\\Desktop\\test.xlsx'
)
xcel = file.active

# Input file header, title, receiver:
headers = ['CÔNG TY TNHH THƯƠNG MẠI DỊCH VẬN TẢI CÁT LINH',
               'MST: 0306980684',
               'Địa chỉ: 441/112 Điện Biên Phủ, P.25, Q.Bình Thạnh, TP.HCM']
title = f'BẢNG THỐNG KÊ VẬN CHUYỂN THÁNG 0{int(datetime.now().month) - 1}/{datetime.now().year}'
receiver = 'Kính gửi: CÔNG TY TNHH LIÊN ĐẠI PHÁT'

for row in range(1, 8):
    if row < 4:
        xcel.cell(row=row, column=1).value = headers[row - 1]
    if row == 5:
        xcel.cell(row=row, column=1).value = title
        xcel.cell(row=row, column=1).alignment = Alignment(horizontal='center')
        xcel.merge_cells('A5:M5')
    if row == 7:
        xcel.cell(row=row, column=1).value = receiver

# Modify the main table:
main_index = 9 + len(LDP['STT']) 
phrase = 'Tổng cộng'

cell = xcel.cell(row=main_index, column=1)
cell.value = phrase
cell.alignment = Alignment(horizontal='center')
xcel.merge_cells(f'A{main_index}:G{main_index}')

# Input file footer:
    # Part I:
footer_index = 9 + len(LDP['STT']) + 2
footers = ['* Cước vận chuyển:', '* Phí nâng:', '* Phí hạ:', '* Phí phát sinh:', '* Tổng cộng:']
for idx in range(len(footers)):
    xcel.cell(row=footer_index + idx, column=1).value = footers[idx]
    xcel.merge_cells(f'A{footer_index + idx}:C{footer_index + idx}')
    
    
    # Part II:
footer_index = footer_index + 6
debts = [f'* Nợ cuối T0{int(datetime.now().month) - 2}/{datetime.now().year}:',
         f'* Phát sinh T0{int(datetime.now().month) - 1}/{datetime.now().year}:',
         f'* Thanh toán T0{int(datetime.now().month) - 1}/{datetime.now().year}:',
         f'* Nợ cuối T0{int(datetime.now().month) - 1}/{datetime.now().year}:']
for idx in range(len(debts)):
    xcel.cell(row=footer_index + idx, column=1).value = debts[idx]
    xcel.merge_cells(f'A{footer_index + idx}:C{footer_index + idx}')

    # Associated values:
footer_index = 9 + len(LDP['STT']) + 2
for col in [8, 9, 11, 12, 13]:
    xcel.cell(row=footer_index, column=4).value = xcel.cell(row=main_index, column=col).value
    footer_index += 1

file.save('C:\\Users\\jio\\Desktop\\test.xlsx')

## Salary table:
- Create main table:

In [ ]:
salary

In [ ]:
# Import salary reference:
salary = pd.read_excel(
    io='C:\\Users\\jio\\Desktop\\CatLinhProject\\Samples\\Bảng lương.xlsx',
    header=0,
)

# Restructure the main content of the table:
salary = df.merge(salary, on='TVC')

columns = ['Kích cỡ', 'Khách hàng ', 'Số xe ', 'Doanh thu VC', 'Tổng cộng (VC)', 'Tổng cộng (NH)', 'Số xe']
salary = salary.drop(columns=columns)

salary['Tổng cộng'] = [0 for idx in range(len(salary['STT']))]
for idx in range(len(salary['STT'])):
    salary.loc[idx, ['Tổng cộng']] = salary.iloc[idx, [8, 10, 11, 12]].sum()

nan_arr = np.array([np.nan]*18)
nan_arr = nan_arr.reshape(9, 2)
salary[['Số lượng dầu cấp', 'Ngày']] = nan_arr

# Add 'Tổng cộng' row:
idx = [i for i in range(len(salary['STT']) + 1)]
salary = salary.reindex(idx)
# salary.loc[16, ['STT']] = '17'
# salary['STT'] = salary['STT'].astype('int8')

salary = salary.sort_values(by='STT', ascending=True)

for col in [7, 8, 10, 11, 12, 13]:
    salary.iloc[9, col] = salary.iloc[:, col].sum()

salary.to_excel(
    'C:\\Users\\jio\\Desktop\\test2.xlsx',
    sheet_name='51C-774.99',
    header=True,
    index=False,
    startrow=4
)

- Add in details:

In [58]:
file = openpyxl.load_workbook('C:\\Users\\jio\\Desktop\\test2.xlsx')
xcel = file.active

# Sheet '51C-774.99' header:
texts = [f'BẢNG LƯƠNG THÁNG {int(datetime.now().month) - 1}/{datetime.now().year}',
         'Số xe: 51C-774.99\t\t\t Họ & tên tài xế: Hà Thanh Phước']
header_idx = 0
for row in [1, 3]:
    xcel.cell(row=row, column=1).value = texts[header_idx]
    if row == 1:
        xcel.cell(row=row, column=1).alignment = Alignment(horizontal='center')
        xcel.merge_cells(f'A{row}:P{row}')
    header_idx += 1

# Sheet '51C-774.99' main table:
main_idx = 5 + len(salary['STT'])
xcel.cell(row=main_idx, column=1).value = 'Tổng cộng'
xcel.cell(row=main_idx, column=1).alignment = Alignment(horizontal='center')
xcel.merge_cells(f'A{main_idx}:G{main_idx}')

# Sheet '51C-774.99' footer:
source = openpyxl.load_workbook('C:\\Users\\jio\\Desktop\\CatLinhProject\\Samples\\Bảng lương T05.xlsx')
sheet = source['51C-774.99']
for row in range(31, 51):
    for col in range(1, 5):
        xcel.cell(row=row, column=col).value = sheet.cell(row=row, column=col).value

file.save('C:\\Users\\jio\\Desktop\\test2.xlsx')

- Copy 'Tổng cộng' sheet:

In [59]:
def copysheet(source_sheet, target_sheet):
    copysheet_attributes(source_sheet, target_sheet)
    copycells(source_sheet, target_sheet)


def copysheet_attributes(source_sheet, target_sheet):
    target_sheet.sheet_format = copy(source_sheet.sheet_format)
    target_sheet.sheet_properties = copy(source_sheet.sheet_properties)
    target_sheet.merged_cells = copy(source_sheet.merged_cells)
    target_sheet.page_margins = copy(source_sheet.page_margins)
    target_sheet.freeze_panes = copy(source_sheet.freeze_panes)

    # Set row dimensions:
    for rn in range(len(source_sheet.row_dimensions)):
        target_sheet.row_dimensions[rn] = copy(source_sheet.row_dimensions[rn])

    if source_sheet.sheet_format.defaultColWidth is None:
        print('Unable to copy default column width')
    else:
        target_sheet.sheet_format.defaultColWidth = \
            copy(source_sheet.sheet_format.defaultColWidth)

    # Set specific column width and hidden property:
    for key, value in source_sheet.column_dimensions.items():
        target_sheet.column_dimensions[key].min = \
            copy(source_sheet.column_dimensions[key].min)
        target_sheet.column_dimensions[key].max = \
            copy(source_sheet.column_dimensions[key].max)
        target_sheet.column_dimensions[key].width = \
            copy(source_sheet.column_dimensions[key].width)
        target_sheet.column_dimensions[key].hidden = \
            copy(source_sheet.column_dimensions[key].hidden)


def copycells(source_sheet, target_sheet):
    for (row, col), source_cell in source_sheet._cells.items():
        target_cell = target_sheet.cell(row=row, column=col)
        target_cell.value = source_cell.value
        target_cell.data_type = source_cell.data_type

        if source_cell.has_style:
            target_cell.font = copy(source_cell.font)
            target_cell.border = copy(source_cell.border)
            target_cell.fill = copy(source_cell.fill)
            target_cell.number_format = copy(source_cell.number_format)
            target_cell.protection = copy(source_cell.protection)
            target_cell.alignment = copy(source_cell.alignment)

        if source_cell.hyperlink:
            target_cell.hyperlink = copy(source_cell.hyperlink)

        if source_cell.comment:
            target_cell.comment = copy(source_cell.comment)

copysheet(source['Tổng cộng'], file.create_sheet('Tổng cộng'))
file.save('C:\\Users\\jio\\Desktop\\test2.xlsx')